In [ ]:
import pandas as pd
import pickle
from collections import namedtuple
from itertools import combinations
import random

In [ ]:
with open("../data/sentiment_texts.pickle", "rb") as f:
    sentiment = pickle.load(f)

with open("../data/mentions_texts.pickle", "rb") as f:
    mentions = pickle.load(f)

In [ ]:
sentiment.drop(
    index=sentiment[sentiment.MessageText.str.len() == 0].index, inplace=True
)

In [ ]:
mentions.drop(columns=["messageid"], inplace=True)
mentions.drop(index=mentions[mentions.MessageText.str.len() == 0].index, inplace=True)
mentions.drop(index=mentions[mentions.issuerid < 0].index, inplace=True)

In [ ]:
ment = mentions[["issuerid", "MessageText"]]
sent = sentiment[["issuerid", "MessageText", "SentimentScore"]]

mrg = pd.merge(ment, sent, on=["issuerid", "MessageText"], how="right")

In [ ]:
mrg = mrg[mrg.SentimentScore > 0]

In [ ]:
mrg

In [ ]:
Sample = namedtuple("Sample", ["message", "issuer_ids", "sentiments"])

In [ ]:
samples = []

for message, group in mrg.groupby("MessageText"):
    samples.append(
        Sample(
            message,
            group["issuerid"].tolist(),
            group["SentimentScore"].tolist(),
        )
    )

In [ ]:
len(samples)

In [ ]:
short_samples = list(filter(lambda x: len(x.message) < 1024, samples))
pairs = list(combinations(short_samples, 2))
pairs = random.sample(pairs, 50_000)

In [ ]:
auged_samples = []

for pair in pairs:
    concat_sample = Sample(
        message=(pair[0].message + " " + pair[1].message),
        issuer_ids=(pair[0].issuer_ids + pair[1].issuer_ids),
        sentiments=(pair[0].sentiments + pair[1].sentiments),
    )
    auged_samples.append(concat_sample)

samples.extend(auged_samples)
random.shuffle(samples)

In [ ]:
samples[-1].issuer_ids, samples[-1].sentiments